## Assignment - 5 5/4/24

## Sentimental Analysis

## Opening the Data Set

In [43]:
import pandas as pd
df = pd.read_json('/Users/puttasathvik/Downloads/TextBasedAnalysis/Assignment/winemag-data-130k-v2.json',lines=False)
df.sample(10)

,points,title,description,taster_name,taster_twitter_handle,price,designation,variety,region_1,region_2,province,country,winery
10731,91,Mantlerhof 2013 Steingraben Reserve Riesling (...,Yeast and earth can be smelled as much as tart...,Anne Krebiehl MW,@AnneInVino,32.0,Steingraben Reserve,Riesling,None,None,Kremstal,Austria,Mantlerhof
106514,87,Château la Roque de By 2010 Médoc,A 50-50 blend of Cabernet Sauvignon and Merlot...,Roger Voss,@vossroger,NaN,None,Bordeaux-style Red Blend,Médoc,None,Bordeaux,France,Château la Roque de By
72037,88,Dopff & Irion 2015 Cuvée René Dopff Pinot Blan...,Gentle friendly notes of yellow mirabelle plum...,Anne Krebiehl MW,@AnneInVino,13.0,Cuvée René Dopff,Pinot Blanc,Alsace,None,Alsace,France,Dopff & Irion
2568,89,Gamache 2011 Estate Grown Malbec (Columbia Val...,Aromas of blue and purple fruit mix with notes...,Sean P. Sullivan,@wawinereport,30.0,Estate Grown,Malbec,Columbia Valley (WA),Columbia Valley,Washington,US,Gamache
16630,93,Monteviejo 2009 Lindaflor La Violeta Malbec (U...,"Black as night in color then lush, ripe and si...",Michael Schachner,@wineschach,111.0,Lindaflor La Violeta,Malbec,Uco Valley,None,Mendoza Province,Argentina,Monteviejo
7195,90,Cockburn's 2015 Bicentenary Vintage (Port),This wine has acidity and juicy black-currant ...,Roger Voss,@vossroger,80.0,Bicentenary Vintage,Port,None,None,Port,Portugal,Cockburn's
7689,90,Domaine La Soufrandise 2012 Vieilles Vignes (...,"Full of bright yellow fruits, this is ripe, ro...",Roger Voss,@vossroger,32.0,Vieilles Vignes,Chardonnay,Pouilly-Fuissé,None,Burgundy,France,Domaine La Soufrandise
3869,91,Clos Pegase 2009 Mitsuko's Vineyard Sauvignon ...,The best Clos Pegase Sauvignon Blanc in years....,None,None,20.0,Mitsuko's Vineyard,Sauvignon Blanc,Carneros,Napa-Sonoma,California,US,Clos Pegase
69241,88,Carol Shelton 2009 Rocky Reserve Florence Vine...,Like your Zins big and in-your-face? This one'...,None,None,35.0,Rocky Reserve Florence Vineyard,Zinfandel,Rockpile,Sonoma,California,US,Carol Shelton
24316,85,Castellargo 2009 Chardonnay (Friuli Grave),"Here's an easy, informal Chardonnay that would...",None,None,17.0,None,Chardonnay,Friuli Grave,None,Northeastern Italy,Italy,Castellargo


## Get the Bing Liu lexicon

In [44]:
from sklearn import preprocessing
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize
print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words in opinion lexicon',
opinion_lexicon.positive()[:10])
print('Examples of negative words in opinion lexicon',
opinion_lexicon.negative()[:10])

Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /Users/puttasathvik/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


## Create the “Dictionary”

In [45]:
# Let's create a dictionary which we can use for scoring our review description
nltk.download('punkt')
df.rename(columns={"reviewText": "description"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}
# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
    word_dict[word] = pos_score
# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
    word_dict[word] = neg_score

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/puttasathvik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Function to do the math

In [46]:
def bing_liu_score(description):
    sentiment_score = 0
    bag_of_words = word_tokenize(description.lower())
    for word in bag_of_words:
        if word in word_dict:
            sentiment_score += word_dict[word]
            return sentiment_score

## Make sure all description are non-empty and are type description then call the function

In [47]:
df['description'].fillna('no review', inplace=True)
df['Bing_Liu_Score'] = df['description'].apply(bing_liu_score)

## Let’s see the output

In [48]:
df[['title',"description", 'Bing_Liu_Score']].head(10)

,title,description,Bing_Liu_Score
0,Nicosia 2013 Vulkà Bianco (Etna),"Aromas include tropical fruit, broom, brimston...",-1.0
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),"This is ripe and fruity, a wine that is smooth...",1.0
2,Rainstorm 2013 Pinot Gris (Willamette Valley),"Tart and snappy, the flavors of lime flesh and...",1.0
3,St. Julian 2013 Reserve Late Harvest Riesling ...,"Pineapple rind, lemon pith and orange blossom ...",-1.0
4,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,"Much like the regular bottling from 2012, this...",1.0
5,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Blackberry and raspberry aromas show a typical...,1.0
6,Terre di Giurfo 2013 Belsito Frappato (Vittoria),"Here's a bright, informal red that opens with ...",1.0
7,Trimbach 2012 Gewurztraminer (Alsace),This dry and restrained wine offers spice in p...,1.0
8,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Savory dried thyme notes accent sunnier flavor...,1.0
9,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,This has great depth of flavor with its fresh ...,1.0


## One more thing:

In [49]:
df.groupby('title').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
title,
1+1=3 2008 Rosé Cabernet Sauvignon (Penedès),1.0
1+1=3 NV Brut Sparkling (Cava),0.0
1+1=3 NV Cygnus Brut Nature Reserva Made With Organic Grapes Sparkling (Cava),1.0
1+1=3 NV Rosé Sparkling (Cava),-1.0
10 Knots 2006 Beachcomber White (Paso Robles),1.0
...,...
àMaurice 2014 Fred Estate Syrah (Walla Walla Valley (WA)),NaN
àMaurice NV Pour Me Red (Columbia Valley (WA)),1.0
Štoka 2009 Izbrani Teran (Kras),1.0
